# TQQQ Trading Strategies - Tutorial Notebook

This notebook demonstrates how to use the TQQQ Trading Strategies Backtesting System.

## Overview

In this tutorial, you'll learn how to:
1. Load and prepare TQQQ data
2. Run backtesting strategies
3. Analyze results
4. Create visualizations
5. Export detailed reports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from trading_strategies import TradingStrategyAnalyzer, load_data_from_csv
from visualization import StrategyVisualizer

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Matplotlib settings
%matplotlib inline
plt.style.use('seaborn-v0_8')

## Step 1: Load Data

First, let's load the TQQQ historical data. You can either:
- Use your own CSV file with OHLCV data
- Generate sample data for testing

In [ ]:
# Option A: Load from CSV file
# data = load_data_from_csv('TQQQ_data.csv')

# Option B: Generate sample data
def generate_sample_data(start_date='2023-01-01', end_date='2024-01-01', initial_price=25.0):
    """Generate sample TQQQ-like data for demonstration."""
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    np.random.seed(42)
    returns = np.random.normal(0.001, 0.03, len(dates))
    prices = initial_price * np.exp(np.cumsum(returns))
    
    df = pd.DataFrame({
        'Open': prices * (1 + np.random.uniform(-0.01, 0.01, len(dates))),
        'High': prices * (1 + np.random.uniform(0, 0.02, len(dates))),
        'Low': prices * (1 - np.random.uniform(0, 0.02, len(dates))),
        'Close': prices,
        'Volume': np.random.randint(1000000, 10000000, len(dates))
    }, index=dates)
    
    return df

# Generate sample data
data = generate_sample_data()
print(f"Loaded {len(data)} days of data")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.head()

## Step 2: Visualize Price Data

Let's take a quick look at the price movement.

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(data.index, data['Close'], linewidth=2, color='steelblue')
plt.fill_between(data.index, data['Low'], data['High'], alpha=0.2, color='steelblue')
plt.title('TQQQ Price Movement', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Basic statistics
print(f"Price Statistics:")
print(f"  Starting Price: ${data['Close'].iloc[0]:.2f}")
print(f"  Ending Price: ${data['Close'].iloc[-1]:.2f}")
print(f"  Min Price: ${data['Low'].min():.2f}")
print(f"  Max Price: ${data['High'].max():.2f}")
print(f"  Buy & Hold Return: {((data['Close'].iloc[-1] - data['Close'].iloc[0]) / data['Close'].iloc[0] * 100):.2f}%")

## Step 3: Initialize Analyzer

Create the backtesting analyzer with starting capital.

In [ ]:
# Initialize with $25,000 starting capital
initial_cash = 25000
analyzer = TradingStrategyAnalyzer(data, initial_cash=initial_cash)

print(f"Analyzer initialized with ${initial_cash:,} starting capital")
print(f"Data period: {analyzer.data.index[0]} to {analyzer.data.index[-1]}")
print(f"Total trading days: {len(analyzer.data)}")

## Step 4: Run Individual Strategies

Let's run each strategy individually and examine the results.

In [ ]:
# Strategy 1: Always Invested
print("Running Strategy 1...")
result_1 = analyzer.strategy_1_buy_on_drop_sell_at_target()

print(f"\nStrategy 1 Results:")
print(f"  Final Value: ${result_1['total_final_value']:,.2f}")
print(f"  Total Return: ${result_1['total_return']:,.2f} ({result_1['return_percentage']:.2f}%)")
print(f"  Completed Trades: {len(result_1['completed_trades'])}")
print(f"  Open Positions: {len(result_1['open_positions'])}")

if result_1['completed_trades']:
    profits = [t.profit_loss for t in result_1['completed_trades']]
    print(f"  Average Trade P/L: ${np.mean(profits):.2f}")
    print(f"  Win Rate: {sum(1 for p in profits if p > 0) / len(profits) * 100:.1f}%")

In [ ]:
# Strategy 2: Tiered Selling
print("Running Strategy 2...")
result_2 = analyzer.strategy_2_tiered_selling()

print(f"\nStrategy 2 Results:")
print(f"  Final Value: ${result_2['total_final_value']:,.2f}")
print(f"  Total Return: ${result_2['total_return']:,.2f} ({result_2['return_percentage']:.2f}%)")
print(f"  Completed Trades: {len(result_2['completed_trades'])}")
print(f"  Open Positions: {len(result_2['open_positions'])}")

In [ ]:
# Strategy 3: Moderate Tiers
print("Running Strategy 3...")
result_3 = analyzer.strategy_3_moderate_tiered()

print(f"\nStrategy 3 Results:")
print(f"  Final Value: ${result_3['total_final_value']:,.2f}")
print(f"  Total Return: ${result_3['total_return']:,.2f} ({result_3['return_percentage']:.2f}%)")
print(f"  Completed Trades: {len(result_3['completed_trades'])}")
print(f"  Open Positions: {len(result_3['open_positions'])}")

## Step 5: Run All Strategies and Compare

Now let's run all strategies together and compare results.

In [ ]:
# Run all strategies
results = analyzer.run_all_strategies()

# Create comparison DataFrame
comparison = []
for key, result in results.items():
    final_val = result.get('total_final_value', result.get('final_value', 0))
    comparison.append({
        'Strategy': result['strategy_name'],
        'Final Value': final_val,
        'Return ($)': result['total_return'],
        'Return (%)': result['return_percentage'],
        'Trades': len(result.get('completed_trades', []))
    })

comparison_df = pd.DataFrame(comparison)
comparison_df = comparison_df.sort_values('Return (%)', ascending=False)
comparison_df

## Step 6: Generate Summary Report

Generate a comprehensive text report.

In [ ]:
analyzer.generate_summary_report()

## Step 7: Create Visualizations

Generate professional charts and dashboards.

In [ ]:
# Create visualizer
viz = StrategyVisualizer(results, data)

# Performance dashboard
viz.create_performance_dashboard(show_plot=True)

In [ ]:
# Equity curve
viz.create_equity_curve(show_plot=True)

In [ ]:
# Profit distribution
viz.create_profit_distribution(show_plot=True)

## Step 8: Analyze Trade Details

Let's examine individual trades from the best performing strategy.

In [ ]:
# Find best strategy
best_key = max(results.items(), key=lambda x: x[1]['return_percentage'])[0]
best_result = results[best_key]

print(f"Analyzing: {best_result['strategy_name']}")
print(f"Return: {best_result['return_percentage']:.2f}%\n")

# Convert trades to DataFrame
if best_result['completed_trades']:
    trades_data = []
    for trade in best_result['completed_trades']:
        trades_data.append({
            'Entry Date': trade.entry_time.strftime('%Y-%m-%d'),
            'Exit Date': trade.exit_time.strftime('%Y-%m-%d'),
            'Entry Price': trade.entry_price,
            'Exit Price': trade.exit_price,
            'P/L ($)': trade.profit_loss,
            'Return (%)': trade.return_pct,
            'Days Held': (trade.exit_time - trade.entry_time).days
        })
    
    trades_df = pd.DataFrame(trades_data)
    print(f"\nFirst 10 Trades:")
    display(trades_df.head(10))
    
    print(f"\nTrade Statistics:")
    print(trades_df.describe())

## Step 9: Export Results

Export detailed results to CSV files for further analysis.

In [ ]:
# Export to CSV
exported_files = analyzer.export_results_to_csv(output_dir='./notebook_results')

print(f"Exported {len(exported_files)} files:")
for filepath in exported_files:
    print(f"  - {filepath}")

## Step 10: Custom Analysis

Perform additional custom analysis on the results.

In [ ]:
# Monthly returns analysis (example)
if best_result['completed_trades']:
    trades_df = pd.DataFrame([{
        'Date': t.exit_time,
        'P/L': t.profit_loss
    } for t in best_result['completed_trades']])
    
    trades_df['Month'] = pd.to_datetime(trades_df['Date']).dt.to_period('M')
    monthly_pnl = trades_df.groupby('Month')['P/L'].sum()
    
    plt.figure(figsize=(14, 6))
    monthly_pnl.plot(kind='bar', color='steelblue')
    plt.title('Monthly Profit/Loss - ' + best_result['strategy_name'], 
              fontsize=14, fontweight='bold')
    plt.xlabel('Month', fontsize=12)
    plt.ylabel('P/L ($)', fontsize=12)
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    plt.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## Conclusion

This notebook demonstrated:
- Loading and preparing data
- Running backtesting strategies
- Analyzing and comparing results
- Creating visualizations
- Exporting detailed reports

### Next Steps

1. Try with your own TQQQ data
2. Adjust strategy parameters
3. Implement custom strategies
4. Perform deeper statistical analysis
5. Optimize strategy parameters

### Resources

- [Full Documentation](../README.md)
- [API Reference](../API_DOCUMENTATION.md)
- [GitHub Repository](https://github.com/yourusername/tqqq-trading-strategies)